# Hist Regressor Code Demo Training Script

Code to show how to train a Scikit learn model on an example dataset.
The code trains a model then pickles it and its preprocessor using joblib.
This is done so that the model can be loaded later for inference.

This code is not intended to be uploaded for BYOM but rather show how to create the necessary artefacts for running BYOM models.

This code has largely been taken from: https://scikit-learn.org/stable/auto_examples/linear_model/plot_poisson_regression_non_normal_loss.html



# TODO

* clipping of data?
* make train/test split, fit and predict
* show a score
* optional features, as a test
  * show a text feature to one-hot
* shap to explain
* show resampling to find uncertainty in a fixed test set

In [2]:
from joblib import dump
from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

print("Fetching dataset")
df = fetch_openml(data_id=41214, as_frame=True).frame

Fetching dataset


In [3]:
# ClaimNb is number of claims
# Frequency is the expected frequency per month
# 
df["Frequency"] = df["ClaimNb"] / df["Exposure"]

In [ ]:
tree_preprocessor = ColumnTransformer(
    [
        (
            "categorical",
            OrdinalEncoder(),
            ["VehBrand", "VehPower", "VehGas", "Region", "Area"],
        ),
        ("numeric", "passthrough", ["VehAge", "DrivAge", "BonusMalus", "Density"]),
    ],
    remainder="drop",
)

inputs = tree_preprocessor.fit_transform(df)


In [4]:
hist_model = HistGradientBoostingRegressor(loss="poisson", max_leaf_nodes=128, random_state=0)

print("Fitting model")
hist_model.fit(inputs, df["Frequency"], sample_weight=df["Exposure"])

# note that after fitting - we'd save a model to disk for future use

Saving preprocessor
Instancing model
Fitting model


HistGradientBoostingRegressor(loss='poisson', max_leaf_nodes=128,
                              random_state=0)

In [5]:
import numpy